In [31]:
import numpy as np
import matplotlib.pyplot as plt
from helper import *
import scipy.sparse as sp
from test import load_data_sparse
from SGD import *
from ALS import *
from Visualization import cv_result
from sklearn.linear_model import LinearRegression
from crossval import *

In [32]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"

ratings, pandas = load_data_sparse(data_name, False)

print(pandas.shape) # (1176952, 3)

stop_criterion = 1e-2
prediction = []

(1176952, 3)


In [33]:
# SGD
nb_feature = 20
lambda_ = 0.004
pred_SGD, _, _ = matrix_factorization_SGD_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using SGD...


In [34]:
# keep only prediction for tests
nonzero_row, nonzero_col = ratings.nonzero()
nonzero_train = list(zip(nonzero_row, nonzero_col))

pred_tmp = []
for i in nonzero_train:
    pred_tmp.append(pred_SGD.item(i))

print(len(pred_tmp)) # (1176952)
print(type(pred_tmp))

prediction = pred_tmp
print(len(prediction))
print(type(prediction))

prediction_save = prediction.copy()

1176952
<class 'list'>
1176952
<class 'list'>


In [35]:
print(len(prediction))

1176952


In [36]:
print(len(pred_tmp))

1176952


In [37]:
# TEST
""""prediction = [] 
prediction = np.asarray(pred_tmp)
print(len(pred_tmp))
print(prediction.shape)
tmp = np.asarray(pred_tmp)
prediction = np.vstack((prediction,tmp))
print(prediction.shape)
print(type(prediction))"""

'"prediction = [] \nprediction = np.asarray(pred_tmp)\nprint(len(pred_tmp))\nprint(prediction.shape)\ntmp = np.asarray(pred_tmp)\nprediction = np.vstack((prediction,tmp))\nprint(prediction.shape)\nprint(type(prediction))'

In [38]:
# ALS
nb_feature = 8
lambda_ = 0.081
pred_ALS, _, _ = ALS_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using ALS...


In [39]:
prediction = prediction_save

In [40]:
print(len(prediction))

1176952


In [41]:
pred_tmp = []
for i in nonzero_train:
    pred_tmp.append(pred_ALS.item(i))

print(len(pred_tmp))  # (1176952) ?

prediction = np.vstack((prediction,pred_tmp))

print(type(prediction))
print(prediction.shape)

1176952
<class 'numpy.ndarray'>
(2, 1176952)


In [42]:
#pandas.head()
y = pandas['Rating']
y.head()

0    4
1    3
2    4
3    3
4    5
Name: Rating, dtype: int64

In [43]:
## Blending function
X = np.transpose(prediction)
#clf = LinearRegression()
#clf = clf.fit(X, y)
#print('Weights of the different models:', clf.coef_)
#clf.score(X, y)

In [44]:
from sklearn.linear_model import Ridge
linreg = Ridge(alpha=0.1, fit_intercept=False)
linreg.fit(X, y)
linreg.coef_

array([0.5692528 , 0.43138293])

In [45]:
linreg.score(X, y)

-0.2165127061344998

In [46]:
print(X[0:10])

[[3.57990296 3.6958964 ]
 [3.98563097 3.58744197]
 [3.40529202 3.37514241]
 [4.09591679 3.89092028]
 [4.14685253 3.89526192]
 [4.44663429 4.44019618]
 [3.89128389 3.98457232]
 [4.1978591  3.90785437]
 [3.81274767 3.47022212]
 [3.65598894 3.66028023]]


In [47]:
print(y[0:10])

0    4
1    3
2    4
3    3
4    5
5    4
6    3
7    3
8    2
9    5
Name: Rating, dtype: int64


In [48]:
# final result
result = X.dot(linreg.coef_)
print(result[0:10])

[3.63221637 3.81639279 3.39445081 4.01008866 4.04095688 4.44668382
 3.9340007  4.07542469 3.66741184 3.66016432]


In [49]:
print(result.shape)

(1176952,)


In [50]:
result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1]

In [ ]:
create_submission_from_prediction(result, output_name)